In [1]:
import math
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import matplotlib.pyplot as plt
import yfinance as yf

from datetime import datetime, timezone
from typing import Optional
from scipy.optimize import brentq
from QuantLib import *

# TODO: ADD PACKAGE VERSIONS TO REQUIREMENTS

In [2]:
# Parameters
TICKER      = "SPY"
MARKET_CAL  = "XNYS"        # Code corresponding to market calendar (e.g. XNYS (NYSE), XNAS (NASDAQ), XLON, XETR)
N_EXP       = 10            # Number of expiries to pull
AS_OF       = None          # Set to 'YYYY-MM-DD' to override (e.g., "2025-10-10")

# Option Chains

Future Developments:

* Substitute bootstrapped curve for r.
 
* Build better dividend/forward curve for q.

In [3]:
def market_close_asof(date_local: pd.Timestamp, cal_code: str = "XNYS"):
    """
    Return the most recent market-close timestamp in UTC following a given market calendar.

    Args:
        date_local (pd.Timestamp): Find the most recent market-close as of this local timestamp.
        cal_code (str): Code corresponding to market calendar, e.g. XNYS (NYSE), XNAS (NASDAQ), XLON, XETR.
    
    Returns:
        pd.Timestamp (tz-aware, UTC)
    """
    
    cal = mcal.get_calendar(cal_code)

    # Build a small date window around date of interest to locate prev/next sessions
    date_utc = date_local.tz_convert("UTC")
    start = (date_utc - pd.Timedelta(days = 10)).date()
    end = (date_utc + pd.Timedelta(days = 10)).date()

    # Full schedule with market_open/market_close in calendar's local timezone
    sched = cal.schedule(start_date = start, end_date = end)

    # Convert schedule times to UTC for robust comparisons
    sched_utc = sched.copy()
    sched_utc["market_open_utc"] = sched_utc["market_open"].dt.tz_convert("UTC")
    sched_utc["market_close_utc"] = sched_utc["market_close"].dt.tz_convert("UTC")
    past_closes = sched_utc[sched_utc["market_close_utc"] <= date_utc]["market_close_utc"]

    # Is the market open?
    date_str = date_utc.date().isoformat()
    date_row = sched_utc.loc[date_str] if date_str in sched_utc.index else None

    # Determine the most recent completed close
    if date_row is not None:
        use_today = (date_utc >= date_row["market_close_utc"])
        if use_today:
            return date_row["market_close_utc"]
        else:
            return past_closes.iloc[-1]
    else:
        return past_closes.iloc[-1]

In [4]:
def continuous_rate_from_annual_percent(pct_yield: float) -> float:
    """Convert a quoted annualized percent yield to continuous compounding."""
    y = float(pct_yield) / 100.0
    return np.log(1.0 + max(y, 0.0))

In [5]:
def exp_option_chain(exp_chain: yf.ticker.Ticker, exp: str, df_rows: list, as_of_date, S, r, q) -> list:

    """
    Aggregate option chain for given ticker at specific expiration.

    Args:
        exp_chain (yf.ticker.Ticker):   Option chain object with two DataFrames, one containing all 
                                        the call options data for the selected expiration date and 
                                        the other all the put options data.
        exp (str): Option expiration date as 'YYYY-MM-DD'.
        df_rows (list): List of option chain DataFrames.
        as_of_date (pd.Timestamp): Today's date.
        S (float): Spot price.
        r (float): Risk-free rate.
        q (float): Dividend yield.

    Returns:
        List of option chain DataFrames with those from exp_chain appended.
    """
    for option_type, option_df in [("call", exp_chain.calls), ("put", exp_chain.puts)]:
        
        if option_df is None or option_df.empty:
            continue
        
        tmp = option_df.copy()
        tmp["type"] = option_type
        tmp["expiry"] = pd.to_datetime(exp, utc=True)
        tmp["date"] = as_of_date
        tmp["underlying"] = S
        tmp["rate"] = r
        tmp["div_yield"] = q

        # Parse lastTradeDate and compute quote age (hours)
        if "lastTradeDate" in tmp.columns:
            tmp["lastTradeDate"] = pd.to_datetime(tmp["lastTradeDate"], utc=True, errors="coerce")
            tmp["quote_age_hours"] = (tmp["date"] - tmp["lastTradeDate"]).dt.total_seconds().clip(lower = 0) / 3600.0
        else:
            tmp["lastTradeDate"] = pd.NaT
            tmp["quote_age_hours"] = np.nan

        tmp.rename(columns={"impliedVolatility": "iv_mkt",
                            "lastPrice": "last"}, inplace=True)
        cols = ["contractSymbol", "date", "expiry", "type", "inTheMoney", "strike", "bid", "ask", "last", "iv_mkt", "underlying", 
                "rate", "div_yield", "volume", "openInterest", "lastTradeDate", "quote_age_hours"]
        tmp = tmp[[c for c in cols if c in tmp.columns]]
        df_rows.append(tmp)
    
    return df_rows

In [6]:
def option_chains(ticker: str, market_cal: str, n_exp: int, as_of: str | None = None) -> pd.DataFrame: 
    """
    Get options chain for given ticker.

    Args:
        ticker (str): Stock ticker symbol.
        market_cal (str): Code corresponding to market calendar, e.g. XNYS (NYSE), XNAS (NASDAQ), XLON, XETR.
        n_exp (int): Number of expiries to include.
        as_of (str | None): Date as 'YYYY-MM-DD'. 

    Returns:
        pd.DataFrame: Option chains for all expirations and option types (calls & puts).
    """
    # Market-close timestamp
    if as_of is None:
        # Most recent market-close as of today
        now_utc = pd.Timestamp.now(tz = "UTC")
        as_of = market_close_asof(now_utc, market_cal)
    else:
        # Market-close on given date
        date_utc = pd.Timestamp(as_of).tz_localize("UTC")
        date_utc = date_utc.normalize() + pd.Timedelta(hours = 23, minutes = 59)
        as_of = market_close_asof(date_utc, market_cal)
    
    # Create ticker object
    tk = yf.Ticker(ticker)

    # Available option expiry dates
    opts = tk.options
    if len(opts) == 0:
        raise RuntimeError(f"No option expiries available from yfinance for {ticker} right now.")
    
    # List of n expiries at least a week out
    time_to_expiry = []
    for expiry_date in opts:
        t = int(max(1, (pd.to_datetime(expiry_date, utc = True) - as_of).days))
        if t >= 7:
            time_to_expiry.append(expiry_date)
    expiries = time_to_expiry[:n_exp]

    # Pull 1 day of price history
    hist = tk.history(period = "1d", end = as_of.strftime("%Y-%m-%d"))
    if hist.empty:
        raise RuntimeError(f"No price history returned for {ticker}.")
    
    # Use latest close as spot
    S0 = float(hist["Close"].iloc[-1])

    # Risk-Free Proxy: 13-Week T-Bill
    try:
        irx = yf.Ticker("^IRX").history(period="5d")["Close"].dropna().iloc[-1]
        r_cont = continuous_rate_from_annual_percent(irx)
    except Exception:
        r_cont = 0.02 # fallback
    
    # Dividend Yield Proxy: trailing 365-day dividends / spot
    try:
        divs = tk.dividends
        recent = divs[divs.index >= (as_of - pd.Timedelta(days=365))]
        # Sum the last 365 days of dividends and divide by spot
        q_cont = float(recent.sum() / S0) if not recent.empty else 0.0 # NOT continuous q but proxy for
    except Exception:
        q_cont = 0.0 # fallback
    
    # Aggregate option chains
    rows = []
    for exp in expiries:
        chain = tk.option_chain(exp)
        rows = exp_option_chain(chain, exp, rows, as_of, S0, r_cont, q_cont)    
    df = pd.concat(rows, ignore_index=True)

    # Compute mid price from bid/ask when both exist; otherwise fall back to last price
    df["mid"] = np.where(np.isfinite(df[["bid", "ask"]]).all(axis = 1), 0.5 * (df["bid"] + df["ask"]), df["last"])
    
    # Compute time to expiry in years using Actual/365
    df["T"] = (pd.to_datetime(df["expiry"]) - pd.to_datetime(df["date"])).dt.days.clip(lower = 0).astype(float) / 365.0

    return df

In [7]:
df = option_chains(TICKER, MARKET_CAL, N_EXP, AS_OF)

In [8]:
print(f"Built df with {len(df)} rows across {df['expiry'].nunique()} expiries. ")
df.head(10)

Built df with 3149 rows across 10 expiries. 


,contractSymbol,date,expiry,type,inTheMoney,strike,bid,ask,last,iv_mkt,underlying,rate,div_yield,volume,openInterest,lastTradeDate,quote_age_hours,mid,T
0,SPY251030C00500000,2025-10-22 20:00:00+00:00,2025-10-30 00:00:00+00:00,call,True,500.0,166.77,170.28,165.82,0.924561,671.289978,0.03718,0.010806,3.0,2.0,2025-10-17 20:12:11+00:00,119.796944,168.525,0.019178
1,SPY251030C00603000,2025-10-22 20:00:00+00:00,2025-10-30 00:00:00+00:00,call,True,603.0,64.04,67.55,60.85,0.526982,671.289978,0.03718,0.010806,1.0,1.0,2025-10-17 14:49:16+00:00,125.178889,65.795,0.019178
2,SPY251030C00605000,2025-10-22 20:00:00+00:00,2025-10-30 00:00:00+00:00,call,True,605.0,62.05,65.56,68.06,0.515019,671.289978,0.03718,0.010806,1.0,1.0,2025-10-20 19:51:37+00:00,48.139722,63.805,0.019178
3,SPY251030C00610000,2025-10-22 20:00:00+00:00,2025-10-30 00:00:00+00:00,call,True,610.0,58.75,58.95,63.66,0.385870,671.289978,0.03718,0.010806,1.0,3.0,2025-10-21 16:35:15+00:00,27.412500,58.850,0.019178
4,SPY251030C00616000,2025-10-22 20:00:00+00:00,2025-10-30 00:00:00+00:00,call,True,616.0,52.81,53.01,51.26,0.357184,671.289978,0.03718,0.010806,2.0,1.0,2025-10-17 18:20:55+00:00,121.651389,52.910,0.019178
5,SPY251030C00635000,2025-10-22 20:00:00+00:00,2025-10-30 00:00:00+00:00,call,True,635.0,34.25,34.42,28.73,0.271492,671.289978,0.03718,0.010806,NaN,3.0,2025-10-16 18:33:44+00:00,145.437778,34.335,0.019178
6,SPY251030C00640000,2025-10-22 20:00:00+00:00,2025-10-30 00:00:00+00:00,call,True,640.0,29.51,29.67,27.27,0.252083,671.289978,0.03718,0.010806,1.0,2.0,2025-10-20 13:55:17+00:00,54.078611,29.590,0.019178
7,SPY251030C00641000,2025-10-22 20:00:00+00:00,2025-10-30 00:00:00+00:00,call,True,641.0,28.56,28.73,26.51,0.248299,671.289978,0.03718,0.010806,6.0,NaN,2025-10-17 17:56:16+00:00,122.062222,28.645,0.019178
8,SPY251030C00644000,2025-10-22 20:00:00+00:00,2025-10-30 00:00:00+00:00,call,True,644.0,25.78,25.97,25.30,0.238472,671.289978,0.03718,0.010806,4.0,2.0,2025-10-17 19:09:40+00:00,120.838889,25.875,0.019178
9,SPY251030C00645000,2025-10-22 20:00:00+00:00,2025-10-30 00:00:00+00:00,call,True,645.0,24.88,25.03,22.94,0.234077,671.289978,0.03718,0.010806,2.0,2.0,2025-10-17 16:53:52+00:00,123.102222,24.955,0.019178


# Clean Data for Calibration

Future Developments:

* Build "stale-ness score" & down-weighting potentially problematic quotes (e.g. very wide spreads, last price far outside book).

In [9]:
TICK = 0.01                 # equity options min tick (adjust if needed)
MIN_OI = 10
AGE_HOURS = 48              # threshold to mark lastTradeDate as stale

In [10]:
def clean_data(df: pd.DataFrame, tick: float = 0.01, min_oi: int = 10, age_hours: int = 48) -> pd.DataFrame:
    """
    Clean data and remove invalid & illiquid quotes.
    
    Args:
        df (pd.DataFrame): Option chains data.
        tick (float): Minimum tick, 0.01 default for equity options. 
        min_oi (int): Minimum open interest to identify illiquid quotes.
        age_hours (int): Threshold to mark lastTradeDate as stale.

    Returns:
        pd.DataFrame
    """
    # Remove rows with non-positive prices/strikes or zero time
    df = df[(df["mid"] > 0) & (df["strike"] > 0) & (df["T"] > 0)]

    # Remove rows with missing IVs, bid or ask quotes
    df = df.dropna(subset = ["iv_mkt", "bid", "ask"])

    # Remove rows with obviously broken IVs: negative/zero, or absurdly high (> 500%)
    df.loc[df["iv_mkt"] <= 0, "iv_mkt"] = np.nan
    df.loc[df["iv_mkt"] > 5.0, "iv_mkt"] = np.nan
    df = df.dropna(subset = ["iv_mkt"]).reset_index(drop = True)

    # Identify invalid quotes: ask < bid, negative bid/ask, or zero book
    df["invalid_quote"] = ((df["ask"] < df["bid"]) | 
                           (df["bid"] < 0) | (df["ask"] <= 0) | 
                           ((df["bid"] == 0) & (df["ask"] == 0)))

    # Identify locked quotes with no liquidity: bid = ask & volume = 0 & very low OI
    df["locked_no_liquidity"] = ((df["ask"] - df["bid"] <= tick + 1e-12) & 
                                 (df["volume"].fillna(0) == 0) & 
                                 (df["openInterest"].fillna(0) < min_oi))
    
    # Identify stale quotes ("too old") that were last traded over AGE_HOURS since as_of
    df["stale_age"] = ((df["quote_age_hours"] > AGE_HOURS))

    # Drop these quotes
    df["drop"] = df[["invalid_quote", "locked_no_liquidity", "stale_age"]].any(axis=1)
    df = df[~df["drop"]]
    df = df.drop(columns=["invalid_quote", "locked_no_liquidity", "stale_age", "drop"])

    return df

In [11]:
df_cal = clean_data(df, TICK, MIN_OI, AGE_HOURS)

In [12]:
print(f"Built df_cal with {len(df_cal)} rows across {df_cal['expiry'].nunique()} expiries. ")
df_cal.head(10)

Built df_cal with 2211 rows across 10 expiries. 


,contractSymbol,date,expiry,type,inTheMoney,strike,bid,ask,last,iv_mkt,underlying,rate,div_yield,volume,openInterest,lastTradeDate,quote_age_hours,mid,T
3,SPY251030C00610000,2025-10-22 20:00:00+00:00,2025-10-30 00:00:00+00:00,call,True,610.0,58.75,58.95,63.66,0.385870,671.289978,0.03718,0.010806,1.0,3.0,2025-10-21 16:35:15+00:00,27.412500,58.850,0.019178
10,SPY251030C00650000,2025-10-22 20:00:00+00:00,2025-10-30 00:00:00+00:00,call,True,650.0,20.40,20.53,20.50,0.216866,671.289978,0.03718,0.010806,10.0,20.0,2025-10-22 16:22:13+00:00,3.629722,20.465,0.019178
11,SPY251030C00653000,2025-10-22 20:00:00+00:00,2025-10-30 00:00:00+00:00,call,True,653.0,17.80,17.92,15.94,0.206917,671.289978,0.03718,0.010806,1.0,10.0,2025-10-22 17:32:42+00:00,2.455000,17.860,0.019178
12,SPY251030C00655000,2025-10-22 20:00:00+00:00,2025-10-30 00:00:00+00:00,call,True,655.0,16.11,16.23,16.22,0.200508,671.289978,0.03718,0.010806,6.0,132.0,2025-10-22 20:14:38+00:00,0.000000,16.170,0.019178
13,SPY251030C00657000,2025-10-22 20:00:00+00:00,2025-10-30 00:00:00+00:00,call,True,657.0,14.46,14.57,12.63,0.193795,671.289978,0.03718,0.010806,1.0,70.0,2025-10-22 17:39:31+00:00,2.341389,14.515,0.019178
14,SPY251030C00658000,2025-10-22 20:00:00+00:00,2025-10-30 00:00:00+00:00,call,True,658.0,13.66,13.76,13.40,0.190560,671.289978,0.03718,0.010806,6.0,64.0,2025-10-22 17:11:25+00:00,2.809722,13.710,0.019178
15,SPY251030C00659000,2025-10-22 20:00:00+00:00,2025-10-30 00:00:00+00:00,call,True,659.0,12.86,12.96,12.95,0.187203,671.289978,0.03718,0.010806,12.0,96.0,2025-10-22 15:37:07+00:00,4.381389,12.910,0.019178
16,SPY251030C00660000,2025-10-22 20:00:00+00:00,2025-10-30 00:00:00+00:00,call,True,660.0,12.08,12.18,11.97,0.184090,671.289978,0.03718,0.010806,23.0,481.0,2025-10-22 18:38:17+00:00,1.361944,12.130,0.019178
17,SPY251030C00661000,2025-10-22 20:00:00+00:00,2025-10-30 00:00:00+00:00,call,True,661.0,11.32,11.41,10.65,0.180794,671.289978,0.03718,0.010806,17.0,63.0,2025-10-22 16:35:07+00:00,3.414722,11.365,0.019178
18,SPY251030C00662000,2025-10-22 20:00:00+00:00,2025-10-30 00:00:00+00:00,call,True,662.0,10.57,10.65,10.51,0.177437,671.289978,0.03718,0.010806,70.0,29.0,2025-10-22 19:32:35+00:00,0.456944,10.610,0.019178


# Calibrate Implied Volatility Surface

In [13]:
def norm_cdf(x):
    """ Standard Normal CDF. """
    return 0.5 * (1.0 + math.erf(x / math.sqrt(2.0)))

def norm_pdf(x): 
    """ Standard Normal PDF. """
    return math.exp(-0.5 * x ** 2) / math.sqrt(2.0 * math.pi)

In [14]:
def bs_price(S: float, K: float, T: float, r: float, q: float, sigma: float, option_type: str = 'c') -> float:
    """
    Compute Black-Scholes option price.

    Args:
        S (float): Spot price.
        K (float): Strike price.
        T (float): Time to expiration.
        r (float): Risk-free rate.
        q (float): Dividend yield.
        sigma (float): Volatility.
        option_type (str): 'c' for call, 'p' for put.
    
    Returns:
        float: Option price.
    """

    if T <= 0 or sigma <= 0 or S <= 0 or K <= 0:
        return max(0.0, (S - K) if option_type == 'c' else (K - S))
    
    sqrt_T = math.sqrt(T)
    d1 = (math.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * sqrt_T)
    d2 = d1 - sigma * sqrt_T

    # Call price
    if option_type == 'c':
        return S * math.exp(-q * T) * norm_cdf(d1) - K * math.exp(-r * T) * norm_cdf(d2)
    # Put price
    else:
        return K * math.exp(-r * T) * norm_cdf(-d2) - S * math.exp(-q * T) * norm_cdf(-d1)

In [15]:
def bs_vega(S: float, K: float, T: float, r: float, q: float, sigma: float) -> float:
    """
    Compute the vega of an option (option price sensitivity to a change in volatility).
    vega(call) = vega(put).

    Args:
        S (float): Spot price.
        K (float): Strike price.
        T (float): Time to expiration.
        r (float): Risk-free rate.
        q (float): Dividend yield.
        sigma (float): Volatility.
    
    Returns:
        float: Option vega.
    """

    if T <= 0 or sigma <= 0 or S <= 0 or K <= 0:
        return 0.0
    sqrt_T = math.sqrt(T)
    d1 = (math.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * sqrt_T)
    return S * sqrt_T * math.exp(-q * T) * norm_pdf(d1)

## Estimate Market Implied Volatility

### Brent's Method with Damped Newton–Raphson Fallback

1. Verify market price lies between the no-arbitrage lower/upper bounds.

2. **Define root function:** $f(\sigma) = BS(S, K, T, r, q, \sigma) - \text{price}$.

3. **Bracketing:** Try to find a bracket $\left[ \sigma_L, \sigma_U \right]$ with $f(\sigma_L) < 0 < f(\sigma_U)$ by expanding $\sigma_U$ repeatedly to try to get a sign change.

4. **Brent's Method:** Guaranteed to converge on a bracket $\left[ \sigma_L, \sigma_U \right]$ with a sign change by mixing bisection, secant, and inverse quadratic interpolation to solve for root.

5. **Damped Newton-Raphson:** If bracketing fails, solve starting from seed $\sigma_{0}$ and iterating until $∣f(\sigma_n)∣$ is below tolerance: $\sigma_{n+1} = \sigma_{n} - \frac{f(\sigma_{n})}{f'(\sigma_{n})}$.

In [16]:
def atm_implied_vol(price: float, S: float, K: float, T: float, r: float = 0.0, q: float = 0.0, option_type: str = 'c') -> float:
    """
    Brenner–Subrahmanyam Approximation for ATM option's implied volatility.
    
    Args:
        price (float): Observed option market price.
        S (float): Spot price.
        K (float): Strike price.
        T (float): Time to expiration.
        r (float): Risk-free rate.
        q (float): Dividend yield.
        option_type (str): 'c' for call, 'p' for put.
    
    Returns:
        float: Implied volatility estimate.
    """

    if T <= 0 or S <= 0 or K <= 0:
        return 0.20  # conservative fallback
    
    # Convert put to synthetic call via put-call parity
    if option_type.lower().startswith('c'):
        C = price
    else: 
        C = price + S * math.exp(-q * T) - K * math.exp(-r * T)
    
    # Brenner–Subrahmanyam ATM IV
    S_discounted = S * math.exp(-q * T)
    if S_discounted <= 0:
        return 0.20
    sigma = math.sqrt(2.0 * math.pi / max(T, 1e-12)) * (C / max(S_discounted, 1e-12))

    # Ensure reasonable value
    return max(1e-6, min(3.0, sigma))

In [17]:
def implied_vol(price: float, S: float, K: float, T: float, r: float = 0.0, q: float = 0.0, option_type: str = 'c',
                sigma_L: float = 1e-6, sigma_U: float = 6.0, tol: float = 1e-8, max_iter: int = 100,
                newton_seed: Optional[float] = None, newton_damp: float = 0.5) -> Optional[float]:
    """
    Numerically solves for market implied volatility using Brent's method. If we can't bracket, fall back on damped Newton-Raphson.

    Args:
        price (float): Observed option market price.
        S (float): Spot price.
        K (float): Strike price.
        T (float): Time to expiration.
        r (float): Risk-free rate.
        q (float): Dividend yield.
        option_type (str): 'c' for call, 'p' for put.
        sigma_L (float): Lower bound for initial volatility bracket (~0%).
        sigma_U (float): Upper bound for initial volatility bracket (~600%).
        tol (float): Tolerance for root-finding accuracy.
        max_iter (int): Maximum number of iterations when solving for sigma.
        newton_seed (Optional[float]): Initial IV seed for Newton-Raphson.
        newton_damp (float): Scaling factor for Newton step.

    Returns:
        float or None if price is outside no-arb bounds or both methods fail.
    """
    
    # Check no-arbitrage price bounds
    K_discounted = K * math.exp(-r * T) # discounted stike
    S_discounted = S * math.exp(-q * T) # dividend-discounted spot
    lower = max(0.0,(S_discounted - K_discounted) if option_type == 'c' else (K_discounted - S_discounted))
    upper = S if option_type == 'c' else K_discounted
    if not (lower - 1e-10 <= price <= upper + 1e-10):
        return None

    # Define root function
    def f(sig): return bs_price(S, K, T, r, q, sig, option_type) - price

    # Bracketing
    f_L, f_U = f(sigma_L), f(sigma_U)
    tries = 0
    # some options (short-dated, deep OTM) can have very high IV --> try to catch this
    while f_L * f_U > 0.0 and sigma_U < 20.0 and tries < 12:
        sigma_U *= 1.5
        f_U = f(sigma_U)
        tries += 1
    
    # Brent's Method (if bracketed)
    try:
        root = brentq(f, sigma_L, sigma_U, xtol = tol, maxiter = max_iter)
        return max(sigma_L, min(sigma_U, root))
    except ValueError:
        pass  # fall through to Newton as a backup

    # Sigma seed for Newton-Raphson
    sigma = (newton_seed if newton_seed is not None else atm_implied_vol(price, S, K, T, r, q, option_type))
    if not (sigma_L <= sigma <= sigma_U):
        sigma = 0.5*(sigma_L + sigma_U)

    # Newton-Raphson (damped, bounded)
    for _ in range(max_iter):
        vega = bs_vega(S, K, T, r, q, sigma)
        f_sigma = f(sigma)
        if abs(f_sigma) < tol:
            return sigma
        if vega <= 1e-14:  # vega too small -> avoid blowing up
            break
        step = f_sigma / vega
        sigma = max(sigma_L, min(sigma_U, sigma - newton_damp * step))
        if abs(step) < tol: # tiny step
            return sigma

    # If we got here, Newton didn't converge
    return None

In [18]:
def get_option_price(bid, ask, last, max_spread_frac = 0.5):
    """
    Determine best option price to estimate market volatility.

    Args:
        bid (float): Bid quote.
        ask (float): Ask quote.
        last (float): Last price
        max_spread_frac (float): Bid-ask spread threshold.
    
    Returns:
        float: Option price to use in later calculations.
    """

    # If bid & ask quotes are present and valid
    if np.isfinite(bid) and np.isfinite(ask) and ask > 0 and bid >= 0 and ask >= bid:
        mid = 0.5*(bid + ask)

        # Reject absurdly wide quotes (e.g. spread > 50% of mid)
        spread_frac = (ask - bid) / mid if mid > 0 else np.inf
        if spread_frac <= max_spread_frac:
            return mid
        # Conservative fallback inside the spread (skews toward liquid side)
        return bid + 0.25 * (ask - bid)
    
    # If only last price is available
    if np.isfinite(last) and last > 0:
        if np.isfinite(bid) and last < bid:
            return bid
        if np.isfinite(ask) and last > ask:
            return ask
        return last
    
    # No usable price
    return np.nan

In [19]:
def chain_implied_vol(df: pd.DataFrame) -> pd.DataFrame:
    """
    Estimate implied volatility for each option using Brent's method (with damped Newton-Raphson fallback).

    Args:
        df (pd.DataFrame): Option chains data.

    Returns:
        pd.DataFrame with "iv_fit" column containing implied volatility estimates.
    """
    
    iv_list = []
    for _, df_row in df.iterrows():
        price_iv = get_option_price(df_row.get("bid", np.nan), df_row.get("ask", np.nan), df_row.get("last", np.nan))
        iv = implied_vol(price = price_iv, 
                         S = df_row["underlying"], 
                         K = df_row["strike"], 
                         T = df_row["T"], 
                         r = df_row["rate"], 
                         q = df_row["div_yield"], 
                         option_type = df_row["type"][0].lower(),
                         newton_seed = df_row["iv_mkt"] if np.isfinite(df_row["iv_mkt"]) else None)
        if iv is None or not np.isfinite(iv):
            iv = df_row["iv_mkt"] if np.isfinite(df_row["iv_mkt"]) else np.nan
        iv_list.append(iv)
    df["iv_fit"] = iv_list

    # Keep only rows we can calibrate to
    df = df.dropna(subset=["iv_fit", "strike", "T"])
    df = df[(df["iv_fit"] > 0) & (df["T"] > 0) & (df["strike"] > 0)]
    df = df[(df["iv_fit"] > 0.01)]
    
    return df

In [20]:
df_cal = chain_implied_vol(df_cal)

In [21]:
print(f"Built df_cal with {len(df_cal)} rows across {df_cal['expiry'].nunique()} expiries. ")
df_cal.head(10)

Built df_cal with 2211 rows across 10 expiries. 


,contractSymbol,date,expiry,type,inTheMoney,strike,bid,ask,last,iv_mkt,underlying,rate,div_yield,volume,openInterest,lastTradeDate,quote_age_hours,mid,T,iv_fit
3,SPY251030C00610000,2025-10-22 20:00:00+00:00,2025-10-30 00:00:00+00:00,call,True,610.0,58.75,58.95,63.66,0.385870,671.289978,0.03718,0.010806,1.0,3.0,2025-10-21 16:35:15+00:00,27.412500,58.850,0.019178,0.385870
10,SPY251030C00650000,2025-10-22 20:00:00+00:00,2025-10-30 00:00:00+00:00,call,True,650.0,20.40,20.53,20.50,0.216866,671.289978,0.03718,0.010806,10.0,20.0,2025-10-22 16:22:13+00:00,3.629722,20.465,0.019178,0.216866
11,SPY251030C00653000,2025-10-22 20:00:00+00:00,2025-10-30 00:00:00+00:00,call,True,653.0,17.80,17.92,15.94,0.206917,671.289978,0.03718,0.010806,1.0,10.0,2025-10-22 17:32:42+00:00,2.455000,17.860,0.019178,0.206917
12,SPY251030C00655000,2025-10-22 20:00:00+00:00,2025-10-30 00:00:00+00:00,call,True,655.0,16.11,16.23,16.22,0.200508,671.289978,0.03718,0.010806,6.0,132.0,2025-10-22 20:14:38+00:00,0.000000,16.170,0.019178,0.200508
13,SPY251030C00657000,2025-10-22 20:00:00+00:00,2025-10-30 00:00:00+00:00,call,True,657.0,14.46,14.57,12.63,0.193795,671.289978,0.03718,0.010806,1.0,70.0,2025-10-22 17:39:31+00:00,2.341389,14.515,0.019178,0.193795
14,SPY251030C00658000,2025-10-22 20:00:00+00:00,2025-10-30 00:00:00+00:00,call,True,658.0,13.66,13.76,13.40,0.190560,671.289978,0.03718,0.010806,6.0,64.0,2025-10-22 17:11:25+00:00,2.809722,13.710,0.019178,0.079650
15,SPY251030C00659000,2025-10-22 20:00:00+00:00,2025-10-30 00:00:00+00:00,call,True,659.0,12.86,12.96,12.95,0.187203,671.289978,0.03718,0.010806,12.0,96.0,2025-10-22 15:37:07+00:00,4.381389,12.910,0.019178,0.094833
16,SPY251030C00660000,2025-10-22 20:00:00+00:00,2025-10-30 00:00:00+00:00,call,True,660.0,12.08,12.18,11.97,0.184090,671.289978,0.03718,0.010806,23.0,481.0,2025-10-22 18:38:17+00:00,1.361944,12.130,0.019178,0.103147
17,SPY251030C00661000,2025-10-22 20:00:00+00:00,2025-10-30 00:00:00+00:00,call,True,661.0,11.32,11.41,10.65,0.180794,671.289978,0.03718,0.010806,17.0,63.0,2025-10-22 16:35:07+00:00,3.414722,11.365,0.019178,0.108640
18,SPY251030C00662000,2025-10-22 20:00:00+00:00,2025-10-30 00:00:00+00:00,call,True,662.0,10.57,10.65,10.51,0.177437,671.289978,0.03718,0.010806,70.0,29.0,2025-10-22 19:32:35+00:00,0.456944,10.610,0.019178,0.112299


## Calibration Weighting

Since not all quotes are equally trustworthy (e.g. options with tight bid-ask spreads are more reliable than illliquid ones, far OTM/ITM options may be more noisy than near-ATM options), we apply the following per-point weights to ensure that less reliable quotes don't overpower or distort the model calibration: 

$$w_i = w_i^{\text{IV}} \cdot w_i^{\text{wing}} = \frac{\exp{(-\lambda \left| k_i \right|)}}{(\Delta \sigma_i)^2}$$

**Inverse-IV-Variance Weights:** $\hspace{5pt} w_i^{\text{IV}} = \frac{1}{(\Delta \sigma_i)^2}$, $\hspace{5pt} \Delta \sigma_i \approx \frac{(\text{bid} - \text{ask})/2}{\text{vega}}$

* Tight Spread $\to$ Low Variance $\to$ High Weight
* Wide Spread $\to$ High Variance $\to$ Low Weight

**Wing Damping Weights:** $\hspace{5pt} w_i^{\text{wing}} = \exp{(-\lambda \left| k_i \right|)}$, $\hspace{5pt} k = \ln(K/F)$

* Down-weight options far from forward, which usually have noisy quotes and very low vega.

**Per-Expiry Normalization:** $\hspace{5pt} \Sigma_{j \in \text{expiry}}w_j^{\text{norm}} = 1$


Future Developments:

* Make inverse-IV-variance weighting more robust for very small IV spreads.

* Determine optimal $\lambda$ for wing damping weights.

In [22]:
LAMBDA_WING = 0.5

In [23]:
def inverse_variance_weighting(bid: float, ask: float, S: float, K: float, T: float, r: float, q: float, iv: float) -> float:
    """
    Give more weight to options with tight bid-ask spreads (as they are more reliable) in model calibration. 
    Convert each bid–ask spread in price space into a half-spread in IV space & assign weights using:
    w = 1 / (spread^2 + epsilon).

    Args:
        bid (float): Bid quote.
        ask (float): Ask quote.
        S (float): Spot price.
        K (float): Strike price.
        T (float): Time to expiration.
        r (float): Risk-free rate.
        q (float): Dividend yield.
        iv (float): Implied volatility.
    
    Returns:
        float    
    """
    
    # Price spread
    half_spread_price = 0.5 * (ask - bid)

    # Convert price spread to IV spread
    vega = bs_vega(S, K, T, r, q, iv)
    if vega > 1e-12:
        iv_spread = half_spread_price / vega
    else:
        return 0.0 # np.nan
    
    # Inverse-variance weight
    return 1.0 / (iv_spread**2 + 1e-12)

In [34]:
def inverse_variance_weighting(bid: float, ask: float, S: float, K: float, T: float, r: float, q: float, iv: float,
                               min_tick: float = 0.01, min_half_spread_iv: float = 0.0025, min_vega: float = 1e-5) -> float:
    """
    Give more weight to options with tight bid-ask spreads (as they are more reliable) in model calibration. 
    Convert each bid–ask spread in price space into a half-spread in IV space & assign weights using:
    w = 1 / (spread^2).

    Args:
        bid (float): Bid quote.
        ask (float): Ask quote.
        S (float): Spot price.
        K (float): Strike price.
        T (float): Time to expiration.
        r (float): Risk-free rate.
        q (float): Dividend yield.
        iv (float): Implied volatility.
        min_tick (float): Tick floor for bid-ask spread.
        min_half_spread_iv (float): Floor for IV half-spread.
        min_vega (float): Vega threshold.
    
    Returns:
        float    
    """

    # Validate
    if not np.isfinite(bid) or not np.isfinite(ask) or ask <= bid or T <= 0 or iv <= 0:
        return np.nan
    
    # Price half-spread with tick floor
    half_spread_price = 0.5 * max(ask - bid, min_tick)

    # Convert price half-spread to IV half-spread
    vega = bs_vega(S, K, T, r, q, iv)
    if vega < min_vega:
        return np.nan
    half_spread_iv = half_spread_price / vega
    half_spread_iv = max(half_spread_iv, min_half_spread_iv)
    
    # Inverse-variance weight
    return 1.0 / (half_spread_iv ** 2)

In [24]:
def wing_damping_weighting(S: float, K: float, T: float, r: float, q: float, wing_lambda: float = 0.5) -> float:
    """
    Down-weight options far from forward to focus calibration on 'core' region where vol smile is anchored using:
    w = exp(-lambda * |k|).
    
    Args:
        S (float): Spot price.
        K (float): Strike price.
        T (float): Time to expiration.
        r (float): Risk-free rate.
        q (float): Dividend yield.
        wing_lambda (float): Strength of exponential damping.
    
    Returns:
        float 
    """

    # Forward price
    F = S * np.exp((r - q) * T)

    # Log-moneyness
    k = np.log(np.maximum(1e-12, (K / np.maximum(1e-12, F))))
    
    # Wing damping: as price moves away from strike, weight decays exponentially
    return np.exp(-wing_lambda * np.abs(k))

In [35]:
def calibration_weighting(df: pd.DataFrame, lambda_wing: float) -> pd.DataFrame:
    """
    Compute weight for each option that indicates how 'trustworthy' the quote is and how much it should 
    influence the volatility surface model calibration. Weights are normalized per expiry so one 
    maturity doesn't overwhelm model fit.

    Args:
        df (pd.DataFrame): Option chains data.
        wing_lambda (float): Rate of exponential decay in wing damping.

    Returns:
        pd.DataFrame containg option chains data with weight columns appended.
    """
    w_iv_list = []
    w_wing_list = []
    for _, df_row in df.iterrows():
        
        bid_w = df_row["bid"]
        ask_w = df_row["ask"]
        S_w = df_row["underlying"]
        K_w = df_row["strike"]
        T_w = df_row["T"]
        r_w = df_row["rate"]
        q_w = df_row["div_yield"]
        iv_w = df_row["iv_fit"]
        
        # Inverse-IV-variance weight
        iv_weight = inverse_variance_weighting(bid_w, ask_w, S_w, K_w, T_w, r_w, q_w, iv_w)
        w_iv_list.append(iv_weight)

        # Wing damping weight
        wd_weight = wing_damping_weighting(S_w, K_w, T_w, r_w, q_w, lambda_wing)
        w_wing_list.append(wd_weight)

    df["w_iv"] = w_iv_list
    df["w_wing"] = w_wing_list
    df["w"] = df["w_iv"] * df["w_wing"]
    df = df.dropna(subset = ["w_iv", "w_wing", "w"]).reset_index(drop = True)

    # Normalize Per Expiry
    df["w_norm"] = 0.0
    for exp, idx in df.groupby("expiry").groups.items():
        idx = list(idx)
        w_sum = df.loc[idx, "w"].sum()
        df.loc[idx, "w_norm"] = df.loc[idx, "w"] / (w_sum if w_sum > 0 else 1.0)
    
    return df

In [36]:
df_cal = calibration_weighting(df_cal, LAMBDA_WING)

In [37]:
print(f"Built df_cal with {len(df_cal)} rows across {df_cal['expiry'].nunique()} expiries. ")
df_cal.head(10)

Built df_cal with 2210 rows across 10 expiries. 


,contractSymbol,date,expiry,type,inTheMoney,strike,bid,ask,last,iv_mkt,...,openInterest,lastTradeDate,quote_age_hours,mid,T,iv_fit,w_iv,w_wing,w,w_norm
0,SPY251030C00610000,2025-10-22 20:00:00+00:00,2025-10-30 00:00:00+00:00,call,True,610.0,58.75,58.95,63.66,0.385870,...,3.0,2025-10-21 16:35:15+00:00,27.412500,58.850,0.019178,0.385870,4866.938790,0.953016,4638.268505,0.000262
1,SPY251030C00650000,2025-10-22 20:00:00+00:00,2025-10-30 00:00:00+00:00,call,True,650.0,20.40,20.53,20.50,0.216866,...,20.0,2025-10-22 16:22:13+00:00,3.629722,20.465,0.019178,0.216866,95977.348698,0.983766,94419.242342,0.005327
2,SPY251030C00653000,2025-10-22 20:00:00+00:00,2025-10-30 00:00:00+00:00,call,True,653.0,17.80,17.92,15.94,0.206917,...,10.0,2025-10-22 17:32:42+00:00,2.455000,17.860,0.019178,0.206917,141622.336254,0.986034,139644.369385,0.007879
3,SPY251030C00655000,2025-10-22 20:00:00+00:00,2025-10-30 00:00:00+00:00,call,True,655.0,16.11,16.23,16.22,0.200508,...,132.0,2025-10-22 20:14:38+00:00,0.000000,16.170,0.019178,0.200508,160000.000000,0.987542,158006.777834,0.008915
4,SPY251030C00657000,2025-10-22 20:00:00+00:00,2025-10-30 00:00:00+00:00,call,True,657.0,14.46,14.57,12.63,0.193795,...,70.0,2025-10-22 17:39:31+00:00,2.341389,14.515,0.019178,0.193795,160000.000000,0.989049,158247.825691,0.008929
5,SPY251030C00658000,2025-10-22 20:00:00+00:00,2025-10-30 00:00:00+00:00,call,True,658.0,13.66,13.76,13.40,0.190560,...,64.0,2025-10-22 17:11:25+00:00,2.809722,13.710,0.019178,0.079650,17022.072210,0.989801,16848.469632,0.000951
6,SPY251030C00659000,2025-10-22 20:00:00+00:00,2025-10-30 00:00:00+00:00,call,True,659.0,12.86,12.96,12.95,0.187203,...,96.0,2025-10-22 15:37:07+00:00,4.381389,12.910,0.019178,0.094833,66778.653661,0.990553,66147.806961,0.003732
7,SPY251030C00660000,2025-10-22 20:00:00+00:00,2025-10-30 00:00:00+00:00,call,True,660.0,12.08,12.18,11.97,0.184090,...,481.0,2025-10-22 18:38:17+00:00,1.361944,12.130,0.019178,0.103147,121149.107859,0.991304,120095.648672,0.006776
8,SPY251030C00661000,2025-10-22 20:00:00+00:00,2025-10-30 00:00:00+00:00,call,True,661.0,11.32,11.41,10.65,0.180794,...,63.0,2025-10-22 16:35:07+00:00,3.414722,11.365,0.019178,0.108640,160000.000000,0.992055,158728.823230,0.008956
9,SPY251030C00662000,2025-10-22 20:00:00+00:00,2025-10-30 00:00:00+00:00,call,True,662.0,10.57,10.65,10.51,0.177437,...,29.0,2025-10-22 19:32:35+00:00,0.456944,10.610,0.019178,0.112299,160000.000000,0.992805,158848.845042,0.008962


## Fit Heston Stochastic Volatility Model (Risk-Neutral Measure)

In [38]:
def fit_heston(df: pd.DataFrame, v0_seed: Optional[float] = None, kappa_seed: float = 1.0, 
               theta_seed: Optional[float] = None, sigma_seed: float = 0.5, rho_seed: float = -0.5) -> tuple:
    """
    Calibrate Heston model to the given option chains.

    Args:
        df (pd.DataFrame): Option chains data.
        v0_seed (Optional[float]): Initial instantaneous variance (volatility squared) seed.
        kappa_seed (float): Mean-reversion speed seed.
        theta_seed (Optional[float]): Long-run mean variance seed.
        sigma_seed (float): Volatility-of-volatility seed.
        rho_seed (float): Correlation with asset price shock seed.

    Returns:
        tuple containing calibrated HestonModel and a list of HestonModelHelpers.
    """

    today = pd.to_datetime(df["date"].iloc[0])
    Settings.instance().evaluationDate = Date(today.day, today.month, today.year)

    # Spot & rate curves (flat proxies)
    spot = float(df["underlying"].iloc[0])
    S_q = QuoteHandle(SimpleQuote(spot))
    r_ts = YieldTermStructureHandle(FlatForward(0, TARGET(), float(df["rate"].iloc[0]), Actual365Fixed()))
    q_ts = YieldTermStructureHandle(FlatForward(0, TARGET(), float(df["div_yield"].iloc[0]), Actual365Fixed()))

    # Build helpers
    helpers, row_idx = [], []
    for i, df_row in df.iterrows():
        # Calendar DAYS to maturity
        T_days = int(max(1, (pd.to_datetime(df_row["expiry"]) - pd.to_datetime(df_row["date"])).days))
        vol_quote = QuoteHandle(SimpleQuote(float(df_row["iv_fit"])))
        strike = float(df_row["strike"])
        # Calibration helper for vanilla option spec
        helper = HestonModelHelper(Period(T_days, Days), TARGET(), spot, strike, vol_quote, r_ts, q_ts)
        helpers.append(helper)
        row_idx.append(i)
    
    # Apply per-point weights (inverse-variance x wing damping x per-expiry norm)
    for helper, i in zip(helpers, row_idx):
        try:
            helper.setWeight(float(df.loc[i, "w_norm"]))
        except Exception:
            pass 
    
    # Seed Heston
    iv_seed = float(np.nanmean(df["iv_fit"])) 
    # iv_seed = iv_seed if iv_seed < 1.0 else iv_seed * 0.01
    v_0 = (v0_seed if v0_seed is not None else max(1e-8, iv_seed**2))
    kappa_0 = kappa_seed
    theta_0 = (theta_seed if theta_seed is not None else v_0)
    sigma_0 = sigma_seed
    rho_0 = rho_seed

    process = HestonProcess(r_ts, q_ts, S_q, v_0, kappa_0, theta_0, sigma_0, rho_0)
    model = HestonModel(process)

    # Engine & link model to helpers
    engine = AnalyticHestonEngine(model)
    for h in helpers:
        h.setPricingEngine(engine)

    # Calibrate (Levenberg–Marquardt)
    method = LevenbergMarquardt()
    end_crit = EndCriteria(1000, 500, 1e-8, 1e-8, 1e-8)
    model.calibrate(helpers, method, end_crit)

    # Return fitted Heston model & helpers
    return model, helpers

In [39]:
atm_mask = (np.abs(np.log(df_cal["strike"]/df_cal["underlying"])) < 0.03)
atm_iv   = float(df_cal.loc[atm_mask, "iv_fit"].median()) if atm_mask.any() else float(df_cal["iv_fit"].median())
v0_seed  = max(1e-4, atm_iv**2)   # floor prevents collapse
theta_seed = np.clip(float(df_cal.groupby("expiry")["iv_fit"].median().median())**2, 1e-4, 0.5)
kappa_seed = 2.0

model, helpers = fit_heston(df_cal, v0_seed = v0_seed, kappa_seed = kappa_seed, theta_seed = theta_seed)

In [40]:
# model, helpers = fit_heston(df_cal)

# Calibrated Heston paramameters
v0_fitted = float(model.v0())
kappa_fitted = float(model.kappa())
theta_fitted = float(model.theta())
sigma_fitted = float(model.sigma())
rho_fitted = float(model.rho())

In [41]:
print("Calibrated Heston params:")
print(f"  v0    = {float(model.v0()):.8f}")
print(f"  kappa = {float(model.kappa()):.8f}")
print(f"  theta = {float(model.theta()):.8f}")
print(f"  sigma = {float(model.sigma()):.8f}")
print(f"  rho   = {float(model.rho()):.8f}")

Calibrated Heston params:
  v0    = 0.00000001
  kappa = 6.29083630
  theta = 0.05189362
  sigma = 1.21678612
  rho   = -0.53844857


In [42]:
# Inspect residuals (vol points)
errs = [h.calibrationError() for h in helpers]
rms = math.sqrt(np.mean(np.square(errs))) if len(errs) else float("nan")
print(f"RMS calibration error (vol pts): {rms:.6f}")

RMS calibration error (vol pts): 0.750489


In [33]:
print("Rows used:", len(df_cal))
print("Sample (S0, r, q):", float(df_cal["underlying"].iloc[0]), float(df_cal["rate"].iloc[0]), float(df_cal["div_yield"].iloc[0]))
print("T stats (years):", df_cal["T"].min(), df_cal["T"].max(), df_cal["T"].median())
print("IV stats:", df_cal["iv_fit"].min(), df_cal["iv_fit"].max(), df_cal["iv_fit"].median())

# Check weights and staleness (if you added those columns)
if "w_norm" in df_cal:
    print("w_norm sum by expiry:\n", df_cal.groupby("expiry")["w_norm"].sum())
if "is_stale" in df_cal:
    print("Stale count:", df_cal["is_stale"].sum())

# Helper sanity
errs = []
bad = []
for j, h in enumerate(helpers):
    e = h.calibrationError()
    errs.append(e)
    if not np.isfinite(e):
        bad.append(j)
print("RMS (raw):", float(np.sqrt(np.mean(np.square(errs)))))
print("Non-finite helper errors:", len(bad), bad[:10])

Rows used: 2211
Sample (S0, r, q): 671.2899780273438 0.03718017215341554 0.010806060327783593
T stats (years): 0.019178082191780823 0.27123287671232876 0.09863013698630137
IV stats: 0.0702368615811031 1.7578137109375 0.21687060677703285
w_norm sum by expiry:
 expiry
2025-10-30 00:00:00+00:00    1.0
2025-10-31 00:00:00+00:00    1.0
2025-11-07 00:00:00+00:00    1.0
2025-11-14 00:00:00+00:00    1.0
2025-11-21 00:00:00+00:00    1.0
2025-11-28 00:00:00+00:00    1.0
2025-12-19 00:00:00+00:00    1.0
2025-12-31 00:00:00+00:00    1.0
2026-01-16 00:00:00+00:00    1.0
2026-01-30 00:00:00+00:00    1.0
Name: w_norm, dtype: float64
RMS (raw): 0.7506200140268886
Non-finite helper errors: 0 []
